In [15]:
import os, sys
import random
import pandas as pd
import shutil
import tarfile

sys.path.insert(0, 'src')
from utils.utils import write_lists, read_lists, ensure_dir

In [8]:
save_dir = os.path.join('saved', 'ADE20K', '0501_105640', 'baseline_explainer')
paths_filename_template = '{}_paths_16_class_freq_attr.txt'
incongruent_paths_path = os.path.join(save_dir, paths_filename_template.format('incongruent'))
congruent_paths_path = os.path.join(save_dir, paths_filename_template.format('congruent'))


seed = 0
n_images = 60

### Select half congruent and half incongruent paths in a shuffled list, mark which are congruent and incongruent and save as csv

In [10]:
incongruent_paths = read_lists(incongruent_paths_path)
congruent_paths = read_lists(congruent_paths_path)

selected = []

random.seed(0)
selected_incongruent = random.sample(incongruent_paths, k=n_images//2)
selected_congruent = random.sample(congruent_paths, k=n_images//2)

selected = selected_congruent + selected_incongruent
random.shuffle(selected)

congruent_flags = []
for path in selected:
    if path in selected_congruent:
        congruent_flags.append(1)
    else:
        congruent_flags.append(0)
        
df = pd.DataFrame({'Path': selected, 'Congruent': congruent_flags})
save_path = os.path.join(save_dir, 'survey_paths_{}.csv'.format(n_images))
if not os.path.exists(save_path):
    df.to_csv(save_path)
    print("Saved image paths to {}".format(save_path))
else:
    print("{} already exists.".format(save_path))

Saved image paths to saved/ADE20K/0501_105640/baseline_explainer/survey_paths.csv


### Read the paths from the csv file and copy files to a local folder, compress folder 

In [16]:
def compress_images(csv_path, save_dir):
    # Obtain paths to images
    df = pd.read_csv(csv_path)
    image_paths = df['Path']
    # Create image save directory
    image_save_dir = os.path.join(save_dir, 'images_{}'.format(n_images))
    ensure_dir(image_save_dir)
    
    # Copy images (noting the index from the list)
    for idx, src_path in enumerate(image_paths):
        image_name = os.path.basename(src_path)
        dst_path = os.path.join(image_save_dir, 'idx{}_{}'.format(idx, image_name))
        
        try:
            shutil.copy(src_path, dst_path)
        except Exception as e:
            print("Unable to copy file from {} to {}: {}".format(src_path, dst_path, e))
            
    # Compress using tar
    tar_path = os.path.join(save_dir, 'survey_images_{}.tar.gz'.format(n_images))
    with tarfile.open(tar_path, "w:gz") as tar:
        tar.add(image_save_dir, arcname=os.path.basename(image_save_dir))
        
save_dir = os.path.join('saved', 'ADE20K', '0501_105640', 'baseline_explainer')
csv_path = os.path.join(save_dir, 'survey_paths_{}.csv'.format(n_images))

compress_images(
    csv_path=csv_path,
    save_dir=save_dir)
